In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
data = pd.read_excel("C:\\Users\\91934\\Desktop\\Aug Hackfest\\Genetic and clinical data.xlsx")

# Encode categorical variables
label_encoder = LabelEncoder()
data['genetic_changes_brca'] = label_encoder.fit_transform(data['genetic_changes_brca'])
data['genetic_changes_rad51'] = label_encoder.fit_transform(data['genetic_changes_rad51'])
data['sub_type'] = label_encoder.fit_transform(data['sub_type'])
data['stage'] = label_encoder.fit_transform(data['stage'])
data['family_history'] = label_encoder.fit_transform(data['family_history'])
# Encode other categorical columns in a similar way...

# Split the dataset into features and target
X = data[['genetic_changes_brca', 'genetic_changes_rad51', 'sub_type', 'stage', 'family_history']]  # Add other features as needed
y = data['outlier']

# Convert labels to binary format
y_binary = y.replace({"Yes": 1, "No": 0})

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)

# Train a CNN model
X_train_reshaped = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

cnn_model = Sequential()
cnn_model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Train a decision tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

# Integration of decision tree and CNN for prediction
def predict_outlier_ensemble(input_data):
    dt_prediction = dt_model.predict(input_data)[0]
    cnn_input = input_data.reshape(1, input_data.shape[1], 1)
    cnn_prediction = np.round(cnn_model.predict(cnn_input))[0][0]
    ensemble_prediction = np.mean([dt_prediction, cnn_prediction])
    return int(ensemble_prediction)

# Example prediction using ensemble
input_data = np.array([1, 0, 1, 0, 1]).reshape(1, 5)
ensemble_prediction = predict_outlier_ensemble(input_data)
print("\033[1m" + f"Ensemble Prediction among Decision Tree and CNN: {ensemble_prediction}" + "\033[0m")


Epoch 1/20


C:\Users\91934\AppData\Local\Temp\ipykernel_8548\3229968125.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_binary = y.replace({"Yes": 1, "No": 0})
c:\Users\91934\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.5526 - loss: 0.6916 - val_accuracy: 0.5267 - val_loss: 0.7153
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5509 - loss: 0.7037 - val_accuracy: 0.4400 - val_loss: 0.7103
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5018 - loss: 0.6881 - val_accuracy: 0.4000 - val_loss: 0.7093
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5190 - loss: 0.6917 - val_accuracy: 0.4067 - val_loss: 0.7085
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5480 - loss: 0.6891 - val_accuracy: 0.4133 - val_loss: 0.7085
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5427 - loss: 0.6872 - val_accuracy: 0.4133 - val_loss: 0.7072
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4709 - loss: 0.6938 - val_accuracy: 0.4467 - val_loss: 0.7060
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5228 - loss: 0.6915 - val_accuracy: 0.4267 - val_loss: 0.7056
Ep

c:\Users\91934\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
